In [18]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from doubt import Boot
from doubt.datasets import (Airfoil, Blog, Concrete, CPU, 
                            FacebookComments, FishBioconcentration,
                            FishToxicity, ForestFire, NewTaipeiHousing,
                            PowerPlant, Protein, Servo,
                            SpaceShuttle)
from mapie.regression import MapieRegressor

## Setting up

In [2]:
def evaluate_nasa(model, X_tr, X_te, y_tr, y_te, uncertainty=0.05):
    n_boots = int(np.sqrt(len(X_tr)))
    
    # Calculate training residuals
    model.fit(X_tr, y_tr)
    tr_preds = model.predict(X_tr)
    te_preds = model.predict(X_te)
    tr_residuals = y_tr - tr_preds
    
    n_train = X_tr.shape[0]
    n_test = X_te.shape[0]

    # Initialise random number generator
    rng = np.random.default_rng(4242)

    # Compute the model variances
    bootstrap_preds = np.empty((n_boots, n_test))
    for boot_idx in range(n_boots):
        train_idxs = rng.choice(range(n_train), size=n_train, replace=True)
        X_btr = X_tr[train_idxs, :]
        y_btr = y_tr[train_idxs]

        model.fit(X_btr, y_btr)
        
        bootstrap_pred = model.predict(X_te)
        bootstrap_preds[boot_idx] = bootstrap_pred

    # Centre the bootstrapped predictions across the bootstrap dimension
    bootstrap_preds = np.mean(bootstrap_preds, axis=0) - bootstrap_preds

    # Add up the bootstrap predictions and the hybrid train/val residuals
    C = np.array([m + o for m in bootstrap_preds for o in tr_residuals])

    # Calculate the intervals
    intervals = np.expand_dims(te_preds, -1) + np.transpose(np.quantile(C, q=[uncertainty/2, 1-uncertainty/2], axis=0))
    
    coverage = np.mean((y_te > intervals[:, 0]) & (y_te < intervals[:, 1]))
    mean_width = np.mean(intervals[:, 1] - intervals[:, 0])
    return coverage, mean_width

In [3]:
def evaluate_doubt(model, X_tr, X_te, y_tr, y_te, uncertainty=0.05):
    n_boots = int(np.sqrt(len(X_tr)))
    
    bmodel = Boot(model, random_seed=4242)
    bmodel.fit(X_tr, y_tr, n_boots=n_boots)
    preds, intervals = bmodel.predict(X_te, uncertainty=uncertainty, n_boots=n_boots)
    
    coverage = np.mean((y_te > intervals[:, 0]) & (y_te < intervals[:, 1]))
    mean_width = np.mean(intervals[:, 1] - intervals[:, 0])
    return coverage, mean_width

In [4]:
def evaluate_mapie(model, X_tr, X_te, y_tr, y_te, uncertainty=0.05):
    bmodel = MapieRegressor(model)
    bmodel.fit(X_tr, y_tr)
    preds, intervals = bmodel.predict(X_te, alpha=uncertainty)
    
    coverage = np.mean((y_te > intervals[:, 0, 0]) & (y_te < intervals[:, 1, 0]))
    mean_width = np.mean(intervals[:, 1] - intervals[:, 0])
    return coverage, mean_width

In [5]:
datasets = []
scaler = StandardScaler()

# Add Doubt datasets
dataset_classes = [
    Airfoil,
    Concrete,
    FishToxicity,
    ForestFire,
    NewTaipeiHousing,
    PowerPlant,
    Protein,
    Servo,
]

for dataset_class in dataset_classes:
    dataset = dataset_class()
    dataset._data = dataset._data.sample(n=min(len(dataset), 2000), random_state=4242)
    X_tr, X_te, y_tr, y_te = dataset.split(test_size=0.1, random_seed=4242)
    X_tr = scaler.fit_transform(X_tr)
    X_te = scaler.transform(X_te)
    datasets.append((dataset_class.__name__, X_tr, X_te, y_tr, y_te))
    
len(datasets)

8

In [6]:
for name, X_tr, X_te, y_tr, y_te in datasets:
    print(f'{name}: {len(X_tr) + len(X_te):,} samples, {X_tr.shape[-1]:,} features')

Airfoil: 1,503 samples, 5 features
Concrete: 1,030 samples, 8 features
FishToxicity: 908 samples, 6 features
ForestFire: 517 samples, 12 features
NewTaipeiHousing: 414 samples, 6 features
PowerPlant: 2,000 samples, 4 features
Protein: 2,000 samples, 9 features
Servo: 167 samples, 4 features


## Linear Regression

In [7]:
model = LinearRegression()

In [8]:
data_dict = defaultdict(list)
for dataset in tqdm(datasets):
    for uncertainty in tqdm(np.arange(0.01, 0.26, 0.01), leave=False):
        nasa_coverage, nasa_mean_width = evaluate_nasa(model, *dataset[1:], uncertainty=uncertainty)
        doubt_coverage, doubt_mean_width = evaluate_doubt(model, *dataset[1:], uncertainty=uncertainty)
        mapie_coverage, mapie_mean_width = evaluate_mapie(model, *dataset[1:], uncertainty=uncertainty)
        data_dict['dataset'].append(dataset[0])
        data_dict['uncertainty'].append(uncertainty)
        data_dict['nasa_coverage_error'].append(100 * (1 - uncertainty - nasa_coverage))
        data_dict['doubt_coverage_error'].append(100 * (1 - uncertainty - doubt_coverage))
        data_dict['mapie_coverage_error'].append(100 * (1 - uncertainty - mapie_coverage))
        data_dict['nasa_mean_width'].append(nasa_mean_width)
        data_dict['doubt_mean_width'].append(doubt_mean_width)
        data_dict['mapie_mean_width'].append(mapie_mean_width)
    
linreg_df = pd.DataFrame(data_dict).set_index(['dataset', 'uncertainty'])
linreg_df

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

nasa_coverage_error  doubt_coverage_error  \
dataset uncertainty                                              
Airfoil 0.01                   -0.280576             -0.280576   
        0.02                    0.158273              0.158273   
        0.03                    1.316547             -0.841727   
        0.04                    1.035971              0.316547   
        0.05                    0.755396              0.035971   
...                                  ...                   ...   
Servo   0.21                   17.461538             17.461538   
        0.22                   16.461538             16.461538   
        0.23                   15.461538             15.461538   
        0.24                   14.461538             14.461538   
        0.25                   13.461538             13.461538   

                     mapie_coverage_error  nasa_mean_width  doubt_mean_width  \
dataset uncertainty                                                            
Airfoil 0.01                     0.438849        28.146645         30.217236   
        0.02                     0.158273        23.710296         23.612272   
        0.03                     0.597122        22.175861         22.217530   
        0.04                     1.035971        20.446465         20.589233   
        0.05                     0.035971        19.354946         19.468813   
...                                   ...              ...               ...   
Servo   0.21                    17.461538         2.698039          2.708471   
        0.22                    16.461538         2.660064          2.664690   
        0.23                    23.153846         2.621453          2.625874   
        0.24                    22.153846         2.588685          2.579588   
        0.25                    21.153846         2.552256          2.536884   

                     mapie_mean_width  
dataset uncertainty                    
Airfoil 0.01                27.872110  
        0.02                23.604344  
        0.03                22.261014  
        0.04                20.682440  
        0.05                19.605997  
...                               ...  
Servo   0.21                 2.485641  
        0.22                 2.435619  
        0.23                 2.374588  
        0.24                 2.349462  
        0.25                 2.289032  

[200 rows x 6 columns]

In [9]:
(linreg_df.abs()
          .describe()
          .loc[['mean', 'std']]
          .T
          .sort_values(by='mean'))

,mean,std
nasa_coverage_error,3.853582,5.380186
doubt_coverage_error,3.916592,4.869778
mapie_coverage_error,4.206815,4.755425
mapie_mean_width,23.972210,31.514170
doubt_mean_width,24.396187,29.597056
nasa_mean_width,24.398568,33.291436


In [10]:
for method in ['mapie', 'doubt', 'nasa']:
    print(method.title())
    for x in linreg_df[f'{method}_coverage_error'].abs():
        print(f'{x:.4f}')
    print()

Mapie
0.4388
0.1583
0.5971
1.0360
0.0360
0.2446
0.9137
0.0863
0.3525
0.7914
0.2086
0.4892
0.7698
1.7698
2.7698
1.6115
0.4532
0.0144
1.1439
0.8633
0.1367
1.1367
1.4173
1.6978
1.9784
1.0000
1.0654
1.1308
0.2617
0.3271
0.3925
0.4766
0.4112
1.2804
0.2804
1.1495
1.0841
1.9533
0.9533
0.8879
0.1121
0.1776
0.6916
0.3084
1.3084
1.3738
2.3738
2.4393
3.4393
3.5047
1.0000
0.9247
0.8495
1.8495
2.8495
2.7742
2.6989
3.6989
4.6989
3.5484
4.5484
3.3978
3.3226
3.2473
4.2473
4.1720
4.0968
5.0968
5.0215
2.7957
3.7957
2.6452
3.6452
1.4194
1.3441
0.9231
0.0769
1.0769
0.1538
0.7692
1.6923
4.5385
3.5385
6.3846
5.3846
6.3077
5.3077
6.2308
5.2308
6.1538
5.1538
8.0000
8.9231
9.8462
8.8462
7.8462
8.7692
7.7692
6.7692
5.7692
1.0000
0.4390
0.5610
0.8780
0.1220
1.1220
0.3171
0.6829
1.6829
2.6829
3.6829
4.6829
3.2439
4.2439
5.2439
6.2439
7.2439
8.2439
9.2439
10.2439
11.2439
12.2439
13.2439
14.2439
15.2439
0.6304
0.1739
0.2826
1.2826
0.1087
0.0217
0.4783
0.3913
0.3043
0.2174
1.2174
1.6739
2.1304
2.5870
2.5000
3.5000
4

## Decision Tree

In [11]:
model = DecisionTreeRegressor()

In [12]:
data_dict = defaultdict(list)
for dataset in tqdm(datasets):
    for uncertainty in tqdm(np.arange(0.01, 0.26, 0.01), leave=False):
        nasa_coverage, nasa_mean_width = evaluate_nasa(model, *dataset[1:], uncertainty=uncertainty)
        doubt_coverage, doubt_mean_width = evaluate_doubt(model, *dataset[1:], uncertainty=uncertainty)
        mapie_coverage, mapie_mean_width = evaluate_mapie(model, *dataset[1:], uncertainty=uncertainty)
        data_dict['dataset'].append(dataset[0])
        data_dict['uncertainty'].append(uncertainty)
        data_dict['nasa_coverage_error'].append(100 * (1 - uncertainty - nasa_coverage))
        data_dict['doubt_coverage_error'].append(100 * (1 - uncertainty - doubt_coverage))
        data_dict['mapie_coverage_error'].append(100 * (1 - uncertainty - mapie_coverage))
        data_dict['nasa_mean_width'].append(nasa_mean_width)
        data_dict['doubt_mean_width'].append(doubt_mean_width)
        data_dict['mapie_mean_width'].append(mapie_mean_width)
    
tree_df = pd.DataFrame(data_dict).set_index(['dataset', 'uncertainty'])
tree_df

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

nasa_coverage_error  doubt_coverage_error  \
dataset uncertainty                                              
Airfoil 0.01                    9.071942             -1.000000   
        0.02                    8.791367             -1.280576   
        0.03                    7.071942             -1.561151   
        0.04                    6.071942             -1.841727   
        0.05                    4.352518             -2.841727   
...                                  ...                   ...   
Servo   0.21                   25.153846              9.769231   
        0.22                   24.153846              8.769231   
        0.23                   23.153846              7.769231   
        0.24                   22.153846              6.769231   
        0.25                   21.153846              5.769231   

                     mapie_coverage_error  nasa_mean_width  doubt_mean_width  \
dataset uncertainty                                                            
Airfoil 0.01                    -1.000000         8.759302         18.826402   
        0.02                    -2.000000         8.867525         15.666897   
        0.03                    -3.000000         8.962252         13.558130   
        0.04                    -4.000000         8.836978         12.650335   
        0.05                    -4.280576         8.929036         11.809594   
...                                   ...              ...               ...   
Servo   0.21                     2.076923         1.166367          1.389733   
        0.22                     1.076923         1.120214          1.345693   
        0.23                     7.769231         1.027899          1.323956   
        0.24                     6.769231         1.120214          1.318664   
        0.25                     5.769231         1.012513          1.307126   

                     mapie_mean_width  
dataset uncertainty                    
Airfoil 0.01                19.686309  
        0.02                16.332165  
        0.03                14.294187  
        0.04                13.319942  
        0.05                12.114827  
...                               ...  
Servo   0.21                 0.887499  
        0.22                 0.830776  
        0.23                 0.620202  
        0.24                 0.617315  
        0.25                 0.532700  

[200 rows x 6 columns]

In [13]:
(tree_df.abs()
        .describe()
        .loc[['mean', 'std']]
        .T
        .sort_values(by='mean'))

,mean,std
doubt_coverage_error,5.136712,3.983757
mapie_coverage_error,7.788081,4.781511
nasa_mean_width,17.756339,26.955159
nasa_coverage_error,20.668546,9.770648
mapie_mean_width,32.817202,62.537580
doubt_mean_width,33.626542,72.941383


In [14]:
for method in ['mapie', 'doubt', 'nasa']:
    print(method.title())
    for x in tree_df[f'{method}_coverage_error'].abs():
        print(f'{x:.4f}')
    print()

Mapie
1.0000
2.0000
3.0000
4.0000
4.2806
5.2806
5.5612
6.5612
7.5612
8.5612
9.5612
10.5612
11.5612
11.8417
12.8417
13.8417
14.1223
15.1223
16.1223
16.4029
16.6835
17.6835
17.2446
18.9640
18.5252
1.0000
2.0000
3.0000
4.0000
5.0000
6.0000
6.0654
8.0000
7.1308
7.1963
9.1308
8.2617
11.1308
9.3271
7.5234
8.5234
8.5888
10.5234
9.6542
10.6542
10.7196
10.7850
12.7196
9.9813
12.8505
0.0753
0.1505
0.8495
1.8495
2.8495
3.8495
3.7742
2.6237
5.7742
5.6989
5.6237
5.5484
6.5484
6.4731
7.4731
9.5484
10.5484
11.5484
11.4731
10.3226
11.3226
10.1720
13.3226
12.1720
12.0968
1.0000
2.0000
3.0000
2.0769
3.0769
4.0769
3.1538
4.1538
0.6154
2.3077
3.3077
2.3846
1.4615
2.4615
3.4615
2.5385
3.5385
4.5385
5.5385
6.5385
5.6154
4.6923
3.7692
6.6923
1.9231
1.0000
0.4390
0.5610
1.5610
2.5610
3.5610
4.5610
3.1220
4.1220
5.1220
6.1220
7.1220
8.1220
9.1220
10.1220
11.1220
12.1220
13.1220
14.1220
15.1220
16.1220
17.1220
15.6829
16.6829
20.1220
1.0000
1.4565
2.4565
2.9130
3.9130
4.9130
5.9130
6.3696
7.3696
7.2826
8.8261
8

## XGBoost

In [21]:
model = XGBRegressor()

In [22]:
data_dict = defaultdict(list)
for dataset in tqdm(datasets):
    for uncertainty in tqdm(np.arange(0.01, 0.26, 0.01), leave=False):
        nasa_coverage, nasa_mean_width = evaluate_nasa(model, *dataset[1:], uncertainty=uncertainty)
        doubt_coverage, doubt_mean_width = evaluate_doubt(model, *dataset[1:], uncertainty=uncertainty)
        mapie_coverage, mapie_mean_width = evaluate_mapie(model, *dataset[1:], uncertainty=uncertainty)
        data_dict['dataset'].append(dataset[0])
        data_dict['uncertainty'].append(uncertainty)
        data_dict['nasa_coverage_error'].append(100 * (1 - uncertainty - nasa_coverage))
        data_dict['doubt_coverage_error'].append(100 * (1 - uncertainty - doubt_coverage))
        data_dict['mapie_coverage_error'].append(100 * (1 - uncertainty - mapie_coverage))
        data_dict['nasa_mean_width'].append(nasa_mean_width)
        data_dict['doubt_mean_width'].append(doubt_mean_width)
        data_dict['mapie_mean_width'].append(mapie_mean_width)
    
xgb_df = pd.DataFrame(data_dict).set_index(['dataset', 'uncertainty'])
xgb_df

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

nasa_coverage_error  doubt_coverage_error  \
dataset uncertainty                                              
Airfoil 0.01                    1.877698             -1.000000   
        0.02                    2.316547             -2.000000   
        0.03                    2.035971             -2.280576   
        0.04                    1.755396             -2.561151   
        0.05                    3.633094             -2.841727   
...                                  ...                   ...   
Servo   0.21                   17.461538              2.076923   
        0.22                   16.461538              1.076923   
        0.23                   23.153846              0.076923   
        0.24                   22.153846             -0.923077   
        0.25                   21.153846             -1.923077   

                     mapie_coverage_error  nasa_mean_width  doubt_mean_width  \
dataset uncertainty                                                            
Airfoil 0.01                    -1.000000         5.137031         14.984581   
        0.02                    -1.280576         4.786507          9.565057   
        0.03                    -1.561151         4.573328          7.715804   
        0.04                    -2.561151         4.399753          7.054147   
        0.05                    -2.122302         4.223996          6.575261   
...                                   ...              ...               ...   
Servo   0.21                     9.769231         1.141633          1.326785   
        0.22                     8.769231         1.139742          1.302527   
        0.23                     7.769231         1.138262          1.282708   
        0.24                     6.769231         1.137033          1.264789   
        0.25                     5.769231         1.135783          1.248879   

                     mapie_mean_width  
dataset uncertainty                    
Airfoil 0.01                13.171167  
        0.02                10.043884  
        0.03                 8.800552  
        0.04                 7.905486  
        0.05                 7.237548  
...                               ...  
Servo   0.21                 0.992993  
        0.22                 0.889391  
        0.23                 0.756648  
        0.24                 0.685062  
        0.25                 0.634748  

[200 rows x 6 columns]

In [23]:
(xgb_df.abs()
       .describe()
       .loc[['mean', 'std']]
       .T
       .sort_values(by='mean'))

,mean,std
doubt_coverage_error,4.861317,3.937940
mapie_coverage_error,5.263642,4.257862
nasa_mean_width,12.691474,21.081309
nasa_coverage_error,20.216097,12.404993
mapie_mean_width,27.333229,64.133766
doubt_mean_width,27.925433,85.991887


In [24]:
for method in ['mapie', 'doubt', 'nasa']:
    print(method.title())
    for x in xgb_df[f'{method}_coverage_error'].abs():
        print(f'{x:.4f}')
    print()

Mapie
1.0000
1.2806
1.5612
2.5612
2.1223
3.1223
3.4029
4.4029
4.6835
4.9640
5.9640
6.9640
7.9640
8.9640
9.2446
10.2446
9.0863
10.0863
11.0863
11.3669
12.3669
12.6475
12.9281
13.9281
11.3309
0.0654
1.0654
2.0654
3.0654
4.0654
5.0654
5.1308
6.1308
7.1308
7.1963
8.1963
9.1963
10.1963
11.1963
12.1963
12.2617
13.2617
13.3271
13.3925
13.4579
13.5234
14.5234
15.5234
15.5888
15.6542
0.0753
0.9247
1.9247
2.9247
3.9247
3.8495
3.7742
2.6237
2.5484
3.5484
4.5484
3.3978
4.3978
5.3978
6.3978
6.3226
6.2473
5.0968
5.0215
6.0215
5.9462
6.9462
4.7204
5.7204
6.7204
1.0000
2.0000
1.0769
2.0769
3.0769
4.0769
3.1538
0.3077
0.6154
1.5385
0.5385
0.4615
1.4615
2.4615
3.4615
4.4615
3.5385
4.5385
1.6923
0.7692
0.1538
1.0769
2.0000
1.0000
0.0000
1.4390
2.8780
1.8780
0.8780
0.1220
1.1220
2.1220
3.1220
4.1220
5.1220
6.1220
7.1220
8.1220
9.1220
10.1220
8.6829
9.6829
10.6829
11.6829
12.6829
13.6829
14.6829
13.2439
14.2439
15.2439
0.4565
0.3696
0.8261
1.2826
1.7391
2.7391
3.1957
3.1087
4.1087
4.0217
3.9348
4.3913
5.39